# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f690ca98490>
├── 'a' --> tensor([[ 0.2169, -0.0195,  0.7124],
│                   [ 0.5592, -0.2965,  0.3944]])
└── 'x' --> <FastTreeValue 0x7f690ca941f0>
    └── 'c' --> tensor([[-0.3561,  0.2147,  0.3066, -0.6032],
                        [-2.5731,  0.8957,  0.0550, -0.6310],
                        [ 1.3175,  0.5634,  0.1722, -0.0203]])

In [4]:
t.a

tensor([[ 0.2169, -0.0195,  0.7124],
        [ 0.5592, -0.2965,  0.3944]])

In [5]:
%timeit t.a

59.4 ns ± 0.0242 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f690ca98490>
├── 'a' --> tensor([[-1.3713,  1.2031, -1.1789],
│                   [ 1.8320,  0.6740,  1.0197]])
└── 'x' --> <FastTreeValue 0x7f690ca941f0>
    └── 'c' --> tensor([[-0.3561,  0.2147,  0.3066, -0.6032],
                        [-2.5731,  0.8957,  0.0550, -0.6310],
                        [ 1.3175,  0.5634,  0.1722, -0.0203]])

In [7]:
%timeit t.a = new_value

58 ns ± 0.0205 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2169, -0.0195,  0.7124],
               [ 0.5592, -0.2965,  0.3944]]),
    x: Batch(
           c: tensor([[-0.3561,  0.2147,  0.3066, -0.6032],
                      [-2.5731,  0.8957,  0.0550, -0.6310],
                      [ 1.3175,  0.5634,  0.1722, -0.0203]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2169, -0.0195,  0.7124],
        [ 0.5592, -0.2965,  0.3944]])

In [11]:
%timeit b.a

58.5 ns ± 0.0145 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.1453e+00,  2.3065e-01,  1.8872e-03],
               [-1.4738e+00,  1.8972e+00,  8.9191e-01]]),
    x: Batch(
           c: tensor([[-0.3561,  0.2147,  0.3066, -0.6032],
                      [-2.5731,  0.8957,  0.0550, -0.6310],
                      [ 1.3175,  0.5634,  0.1722, -0.0203]]),
       ),
)

In [13]:
%timeit b.a = new_value

493 ns ± 0.203 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

837 ns ± 2.03 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 29.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 161 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 212 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f690caa1cd0>
├── 'a' --> tensor([[[ 0.2169, -0.0195,  0.7124],
│                    [ 0.5592, -0.2965,  0.3944]],
│           
│                   [[ 0.2169, -0.0195,  0.7124],
│                    [ 0.5592, -0.2965,  0.3944]],
│           
│                   [[ 0.2169, -0.0195,  0.7124],
│                    [ 0.5592, -0.2965,  0.3944]],
│           
│                   [[ 0.2169, -0.0195,  0.7124],
│                    [ 0.5592, -0.2965,  0.3944]],
│           
│                   [[ 0.2169, -0.0195,  0.7124],
│                    [ 0.5592, -0.2965,  0.3944]],
│           
│                   [[ 0.2169, -0.0195,  0.7124],
│                    [ 0.5592, -0.2965,  0.3944]],
│           
│                   [[ 0.2169, -0.0195,  0.7124],
│                    [ 0.5592, -0.2965,  0.3944]],
│           
│                   [[ 0.2169, -0.0195,  0.7124],
│                    [ 0.5592, -0.2965,  0.3944]]])
└── 'x' --> <FastTreeValue 0x7f690caa1b50>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 37.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f690caa1d00>
├── 'a' --> tensor([[ 0.2169, -0.0195,  0.7124],
│                   [ 0.5592, -0.2965,  0.3944],
│                   [ 0.2169, -0.0195,  0.7124],
│                   [ 0.5592, -0.2965,  0.3944],
│                   [ 0.2169, -0.0195,  0.7124],
│                   [ 0.5592, -0.2965,  0.3944],
│                   [ 0.2169, -0.0195,  0.7124],
│                   [ 0.5592, -0.2965,  0.3944],
│                   [ 0.2169, -0.0195,  0.7124],
│                   [ 0.5592, -0.2965,  0.3944],
│                   [ 0.2169, -0.0195,  0.7124],
│                   [ 0.5592, -0.2965,  0.3944],
│                   [ 0.2169, -0.0195,  0.7124],
│                   [ 0.5592, -0.2965,  0.3944],
│                   [ 0.2169, -0.0195,  0.7124],
│                   [ 0.5592, -0.2965,  0.3944]])
└── 'x' --> <FastTreeValue 0x7f690caa15e0>
    └── 'c' --> tensor([[-0.3561,  0.2147,  0.3066, -0.6032],
                        [-2.5731,  0.8957,  0.0550, -0.6310],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 91.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.1 µs ± 223 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.2169, -0.0195,  0.7124],
                [ 0.5592, -0.2965,  0.3944]],
       
               [[ 0.2169, -0.0195,  0.7124],
                [ 0.5592, -0.2965,  0.3944]],
       
               [[ 0.2169, -0.0195,  0.7124],
                [ 0.5592, -0.2965,  0.3944]],
       
               [[ 0.2169, -0.0195,  0.7124],
                [ 0.5592, -0.2965,  0.3944]],
       
               [[ 0.2169, -0.0195,  0.7124],
                [ 0.5592, -0.2965,  0.3944]],
       
               [[ 0.2169, -0.0195,  0.7124],
                [ 0.5592, -0.2965,  0.3944]],
       
               [[ 0.2169, -0.0195,  0.7124],
                [ 0.5592, -0.2965,  0.3944]],
       
               [[ 0.2169, -0.0195,  0.7124],
                [ 0.5592, -0.2965,  0.3944]]]),
    x: Batch(
           c: tensor([[[-0.3561,  0.2147,  0.3066, -0.6032],
                       [-2.5731,  0.8957,  0.0550, -0.6310],
                       [ 1.3175,  0.5634,  0.1722, -0.0203]],
         

In [26]:
%timeit Batch.stack(batches)

77.5 µs ± 121 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.2169, -0.0195,  0.7124],
               [ 0.5592, -0.2965,  0.3944],
               [ 0.2169, -0.0195,  0.7124],
               [ 0.5592, -0.2965,  0.3944],
               [ 0.2169, -0.0195,  0.7124],
               [ 0.5592, -0.2965,  0.3944],
               [ 0.2169, -0.0195,  0.7124],
               [ 0.5592, -0.2965,  0.3944],
               [ 0.2169, -0.0195,  0.7124],
               [ 0.5592, -0.2965,  0.3944],
               [ 0.2169, -0.0195,  0.7124],
               [ 0.5592, -0.2965,  0.3944],
               [ 0.2169, -0.0195,  0.7124],
               [ 0.5592, -0.2965,  0.3944],
               [ 0.2169, -0.0195,  0.7124],
               [ 0.5592, -0.2965,  0.3944]]),
    x: Batch(
           c: tensor([[-0.3561,  0.2147,  0.3066, -0.6032],
                      [-2.5731,  0.8957,  0.0550, -0.6310],
                      [ 1.3175,  0.5634,  0.1722, -0.0203],
                      [-0.3561,  0.2147,  0.3066, -0.6032],
                      [-2.5731,  

In [28]:
%timeit Batch.cat(batches)

145 µs ± 321 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

307 µs ± 907 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
